<a href="https://colab.research.google.com/github/Larsdegroot/med_Unet/blob/main/run_experiment_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run UNet Training
This notebook is meant to be used in google colab to train a UNet model on the WMH dataset for MRI brain image segmentation.

## Set up environment

In [1]:
# don't restart session it's fine colab is a liar
!pip3 install torch torchvision torchaudio
!pip3 install "lightning[pytorch-extra]"
!pip3 install monai
!pip3 install SimpleITK

In [3]:
! git clone "https://github.com/Larsdegroot/med_Unet"

Cloning into 'med_Unet'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 185 (delta 6), reused 2 (delta 0), pack-reused 159 (from 1)
Receiving objects: 100% (185/185), 1.24 MiB | 4.83 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [5]:
import zipfile
from pathlib import Path

# Define the target directory
target_dir = Path("med_Unet/data")

# Create the directory if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)

# Extract the ZIP files
for dataset in ["WMH", "BraTS"]:
  try:
    with zipfile.ZipFile(f"med_Unet/data/{dataset}.zip", 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"Extracted the {dataset} dataset.")
  except FileNotFoundError: print(f"No zipfile found for the {dataset} dataset in {target_dir}")
  except zipfile.BadZipFile: print(f"Zipfile for {dataset} not complete, wait for it to upload")


No zipfile found for the WMH dataset in med_Unet/data
No zipfile found for the BraTS dataset in med_Unet/data


## Config rclone

In [ ]:
!wget https://downloads.rclone.org/v1.68.2/rclone-v1.68.2-linux-amd64.deb
!apt install ./rclone-v1.68.2-linux-amd64.deb
!sudo apt-get -y install fuse3

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/onedrive

In [ ]:
!nohup rclone mount onedrive:/ /content/onedrive --daemon --vfs-cache-mode full --log-file /tmp/rclone.log --log-level DEBUG

## Run training

In [2]:
# run exp on Colab
!python3 /content/med_Unet/main.py fit --config /content/med_Unet/configs/configs_exp_7_1_brats.yaml

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


In [5]:
# run exp locally
!python ./main.py fit --config ./configs/configs_exp_7_brats.yaml

c:\Users\samar\Desktop\Samar\Studies\MSc\BiBC\AIMI\venv\Lib\site-packages\ignite\handlers\checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
No seed found, seed set to 0
Seed set to 0
`LitUNet.configure_optimizers` will be overridden by `LightningCLI.configure_optimizers`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Traceback (most recent call last):
  File "c:\Users\samar\Desktop\Samar\Studies\MSc\BiBC\AIMI\med_Unet\main.py", line 23, in <module>
    cli_main()
  File "c:\Users\samar\Desktop\Samar\Studies\MSc\BiBC\AIMI\med_Unet\main.py", line 19, in cli_main
    cli = LightningCLI(LitUNet, MRIDataModule)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samar\Desktop\Samar\Studies\MSc\BiBC\AIMI\

## Run Testing

In [ ]:
#!python3 med_Unet/main.py test --config med_Unet/configs/testing_config